In [3]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder

# 1. Load master dataset
file_path = "vegetable_yield_master_jan2022_sep2025.xlsx"
df = pd.read_excel(file_path)

# 2. Convert Month_Year to datetime
df["Month_Year"] = pd.to_datetime(df["Month_Year"], format="%Y-%m")

# 3. Extract Year and Month
df["Year"] = df["Month_Year"].dt.year
df["Month"] = df["Month_Year"].dt.month

# 4. Add Season (Sri Lanka)
# Maha: Oct–Mar, Yala: Apr–Sep
def get_season(month):
    if month in [10, 11, 12, 1, 2, 3]:
        return "Maha"
    else:
        return "Yala"

df["Season"] = df["Month"].apply(get_season)

# 5. Encode Product Name
le = LabelEncoder()
df["Product_Code"] = le.fit_transform(df["Product_Name"])

# 6. Sort correctly for time-series
df = df.sort_values(["Product_Name", "Month_Year"])

# 7. Create lag features (per vegetable)
df["Yield_lag_1"] = df.groupby("Product_Name")["Yield_ha"].shift(1)
df["Yield_lag_2"] = df.groupby("Product_Name")["Yield_ha"].shift(2)
df["Yield_lag_3"] = df.groupby("Product_Name")["Yield_ha"].shift(3)

# 8. Drop rows with missing lag values
df_ml = df.dropna().reset_index(drop=True)

# 9. Save ML-ready dataset
output_path = "vegetable_yield_ml_ready_step1.xlsx"
df_ml.to_excel(output_path, index=False)

print("STEP 1 completed!")
print("ML dataset saved as:", output_path)
print("Final shape:", df_ml.shape)


STEP 1 completed!
ML dataset saved as: vegetable_yield_ml_ready_step1.xlsx
Final shape: (600, 10)


In [4]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_absolute_error, mean_squared_error
import numpy as np

# 1. Load ML-ready dataset
df = pd.read_excel("vegetable_yield_ml_ready_step1.xlsx")

# 2. Encode Season (Maha=0, Yala=1)
df["Season"] = df["Season"].map({"Maha": 0, "Yala": 1})

# 3. Select features and target
FEATURES = [
    "Year",
    "Month",
    "Season",
    "Product_Code",
    "Yield_lag_1",
    "Yield_lag_2",
    "Yield_lag_3"
]

TARGET = "Yield_ha"

X = df[FEATURES]
y = df[TARGET]

# 4. Time-based train-test split (IMPORTANT)
# Train = first 80%, Test = last 20%
split_index = int(len(df) * 0.8)

X_train = X.iloc[:split_index]
X_test  = X.iloc[split_index:]
y_train = y.iloc[:split_index]
y_test  = y.iloc[split_index:]

# 5. Train Random Forest model
rf_model = RandomForestRegressor(
    n_estimators=300,
    max_depth=12,
    min_samples_split=5,
    random_state=42,
    n_jobs=-1
)

rf_model.fit(X_train, y_train)

# 6. Predictions
y_pred = rf_model.predict(X_test)

# 7. Evaluation
mae = mean_absolute_error(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))

print("✅ Random Forest Training Completed")
print(f"MAE  : {mae:.2f} ha")
print(f"RMSE : {rmse:.2f} ha")

# 8. Feature importance
feature_importance = pd.DataFrame({
    "Feature": FEATURES,
    "Importance": rf_model.feature_importances_
}).sort_values(by="Importance", ascending=False)

print("\n📊 Feature Importance:")
print(feature_importance)


✅ Random Forest Training Completed
MAE  : 197.93 ha
RMSE : 255.87 ha

📊 Feature Importance:
        Feature  Importance
4   Yield_lag_1    0.708753
1         Month    0.171637
6   Yield_lag_3    0.059561
5   Yield_lag_2    0.021752
2        Season    0.014398
3  Product_Code    0.014330
0          Year    0.009568


In [2]:
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestRegressor



# -------------------------------
# 1. Load ML-ready dataset
# -------------------------------
df = pd.read_excel("vegetable_yield_ml_ready_step1.xlsx")

# Encode Season
df["Season"] = df["Season"].map({"Maha": 0, "Yala": 1})

FEATURES = [
    "Year",
    "Month",
    "Season",
    "Product_Code",
    "Yield_lag_1",
    "Yield_lag_2",
    "Yield_lag_3"
]

TARGET = "Yield_ha"

X = df[FEATURES]
y = df[TARGET]



# -------------------------------
# 2. Train model on FULL data
# (important for future prediction)
# -------------------------------
rf_model = RandomForestRegressor(
    n_estimators=300,
    max_depth=12,
    min_samples_split=5,
    random_state=42,
    n_jobs=-1
)

rf_model.fit(X, y)



# -------------------------------
# 3. Prepare next-month data
# -------------------------------
last_month = df["Month_Year"].max()
next_month = last_month + pd.DateOffset(months=1)

year = next_month.year
month = next_month.month

# Season logic
season = 0 if month in [10,11,12,1,2,3] else 1  # Maha=0, Yala=1

future_rows = []

for product in df["Product_Name"].unique():
    product_df = df[df["Product_Name"] == product].sort_values("Month_Year")

    # Get last 3 yields
    lag1 = product_df.iloc[-1]["Yield_ha"]
    lag2 = product_df.iloc[-2]["Yield_ha"]
    lag3 = product_df.iloc[-3]["Yield_ha"]

    product_code = product_df.iloc[-1]["Product_Code"]

    future_rows.append({
        "Product_Name": product,
        "Year": year,
        "Month": month,
        "Season": season,
        "Product_Code": product_code,
        "Yield_lag_1": lag1,
        "Yield_lag_2": lag2,
        "Yield_lag_3": lag3
    })

future_df = pd.DataFrame(future_rows)

# -------------------------------
# 4. Predict
# -------------------------------
future_df["Predicted_Yield_ha"] = rf_model.predict(future_df[FEATURES])



# -------------------------------
# 5. Save predictions
# -------------------------------
output_file = "vegetable_yield_prediction_oct_2025.xlsx"
future_df[["Product_Name", "Predicted_Yield_ha"]].to_excel(output_file, index=False)

print("✅ STEP 3 completed")
print("Prediction month:", next_month.strftime("%B %Y"))
print("Predictions saved to:", output_file)


✅ STEP 3 completed
Prediction month: October 2025
Predictions saved to: vegetable_yield_prediction_oct_2025.xlsx
